---

### 1. **Basic Logger**

**Description:**
Write a function decorator `log_run` that prints `Running <function_name>` before the function runs and `Finished <function_name>` after it completes.

---

In [21]:

def log_run(func):
    def wrapper(*args, **kwargs):
        print(f'Running {func.__name__}')
        result = func(*args, **kwargs)
        print(f'Finished {func.__name__}')
        return result
    return wrapper
    
@log_run
def name():
    print('My name is Mahbub')


In [4]:
name()

Running name
My name is Mahbub
Finished name


### *args VS **kwargs

Both `*args` and `**kwargs` are used in Python to handle variable numbers of arguments in functions, but they serve different purposes:

### **1. `*args` (Non-Keyword Arguments)**
- Stands for **"arguments"** (you can use any name after `*`, but `args` is the convention).
- Used to pass a **variable number of positional arguments** to a function.
- Collects extra positional arguments into a **tuple**.

#### **Example:**
```python
def sum_numbers(*args):
    total = 0
    for num in args:
        total += num
    return total

print(sum_numbers(1, 2, 3))  # Output: 6
print(sum_numbers(10, 20))    # Output: 30
```
- Inside the function, `args` is a **tuple** containing all extra positional arguments.

---

### **2. `**kwargs` (Keyword Arguments)**
- Stands for **"keyword arguments"** (again, any name works, but `kwargs` is standard).
- Used to pass a **variable number of keyword arguments** (like `name="Alice"`).
- Collects extra keyword arguments into a **dictionary**.

#### **Example:**
```python
def print_info(**kwargs):
    for key, value in kwargs.items():
        print(f"{key}: {value}")

print_info(name="Alice", age=25, city="New York")
```
**Output:**
```
name: Alice
age: 25
city: New York
```
- Inside the function, `kwargs` is a **dictionary** containing all extra keyword arguments.

---

### **Key Differences**
| Feature       | `*args` | `**kwargs` |
|--------------|---------|------------|
| **Purpose**  | Handles extra positional arguments | Handles extra keyword arguments |
| **Type**     | Collects into a **tuple** | Collects into a **dictionary** |
| **Usage**    | `func(1, 2, 3)` → `args = (1, 2, 3)` | `func(a=1, b=2)` → `kwargs = {'a':1, 'b':2}` |
| **Syntax**   | `def func(*args)` | `def func(**kwargs)` |

---

### **When to Use Both Together?**
You can combine them to accept any mix of arguments:
```python
def example_function(arg1, arg2, *args, **kwargs):
    print("Fixed args:", arg1, arg2)
    print("Extra positional args (tuple):", args)
    print("Extra keyword args (dict):", kwargs)

example_function(10, 20, 30, 40, name="Bob", age=30)
```
**Output:**
```
Fixed args: 10 20
Extra positional args (tuple): (30, 40)
Extra keyword args (dict): {'name': 'Bob', 'age': 30}
```

### **Summary**
- `*args` → For **variable positional arguments** (stored in a **tuple**).
- `**kwargs` → For **variable keyword arguments** (stored in a **dictionary**).
- Both make functions more flexible when you don’t know the exact number of arguments in advance.

Would you like a practical example where both are used in a decorator? 😊

---

### 2. **Execution Timer**

**Description:**
Create a decorator `time_it` that calculates and prints the **execution time** of a function simulating a time-consuming data operation.

---

In [7]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs)  
        end_time = time.time()  
        elapsed_time = end_time - start_time  
        print(f"Function '{func.__name__}' took {elapsed_time:.4f} seconds to run.")
        return result
    return wrapper


@time_it
def simulate_data_operation(size):
    data = [i ** 2 for i in range(size)]  
    return len(data)

# Test
simulate_data_operation(1_000_000)  

Function 'simulate_data_operation' took 0.3147 seconds to run.


1000000

---

### 3. **Ignore None Returns**

**Description:**
Write a decorator `skip_if_none` that **skips execution** if any argument passed to the function is `None`.

---

In [12]:
def skip_if_none(func):
    def wrapper(*args, **kwargs):
        if any(arg is None for arg in args):
            print(f"Skipping '{func.__name__}': Positional argument is None.")
            return None
        
        if any(value is None for value in kwargs.values()):
            print(f"Skipping '{func.__name__}': Keyword argument is None.")
            return None
        
        return func(*args, **kwargs)
    return wrapper


@skip_if_none
def process_data(data, prefix="", suffix=""):
    
    return f"{prefix}{data}{suffix}"



In [13]:
print(process_data("Hello"))                  
print(process_data("Hello", prefix="[ "))      
print(process_data(None))                      
print(process_data("Hi", suffix=None))        

Hello
[ Hello
Skipping 'process_data': Positional argument is None.
None
Skipping 'process_data': Keyword argument is None.
None


In [18]:
def safe_run(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)  
        except Exception as e:
            print(f"Exception occurred in '{func.__name__}': {str(e)}")
            return -1  
    return wrapper


@safe_run
def risky_operation(x, y):
    return x / y 



In [20]:

print(risky_operation(10, 2))   
print(risky_operation(10, 0))   

5.0
Exception occurred in 'risky_operation': division by zero
-1


---

### 5. **Retry Decorator (Fixed Attempts)**

**Description:**
Create a `retry` decorator that **retries a failing function 3 times** (simulate transient DB/network issues).

---

In [2]:
import time
from functools import wraps

def retry(max_attempts=3, delay=1, backoff=2):
    
    def decorator(func):
        @wraps(func)  # Preserves function metadata (e.g., __name__, docstring)
        def wrapper(*args, **kwargs):
            attempt = 1
            current_delay = delay
            while attempt <= max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_attempts:
                        raise  
                    print(f" Attempt {attempt}/{max_attempts} failed: {str(e)}. Retrying in {current_delay}s...")
                    time.sleep(current_delay)
                    current_delay *= backoff  
                    attempt += 1
        return wrapper
    return decorator


@retry(max_attempts=3, delay=1, backoff=2)
def connect_to_database():
    import random
    if random.random() < 0.7:  # 70% chance of failure
        raise ConnectionError("Database connection failed!")
    return "Connected successfully!"

# Test
print(connect_to_database())  

 Attempt 1/3 failed: Database connection failed!. Retrying in 1s...
 Attempt 2/3 failed: Database connection failed!. Retrying in 2s...


ConnectionError: Database connection failed!